In [2]:
import pandas as pd
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn.linear_model import LogisticRegression

raw_data = pd.read_excel("Universalbank_1500.xlsx")
ub_df = pd.DataFrame(raw_data)
ub_df = ub_df.drop(["ZIP Code", "ID"], axis=1)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:

model = LinearDiscriminantAnalysis()
model.fit(X, y)
return model

NameError: name 'X' is not defined

In [ ]:

model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X, y)
return model

In [ ]:
for colname, coldata in X.iteritems():
    if sorted(coldata.unique()) != [0, 1]:
        output = " Mean of {}: {}\n Standard Deviation of {}: {}".format(colname, coldata.mean(), colname,
                                                                         coldata.std())
        print(output)
    else:
        prob = coldata.value_counts()[1] / len(coldata)
        output = " Probability of accepting loan if {} is True: {}".format(colname, prob)
        print(output)

In [ ]:
cols = X.drop("Personal Loan", axis=1).columns
sum1 = pd.DataFrame(columns=cols)
sum0 = pd.DataFrame(columns=cols)
s1, s2 = 0, 0
for i, row in X.iterrows():
    if row["Personal Loan"] == 1:
        sum1.loc[s1] = row.drop("Personal Loan")
        s1 += 1
    else:
        sum0.loc[s2] = row.drop("Personal Loan")
        s2 += 1
print("For accepted Personal Loan: ")
computesummary(sum1)
print("For rejected Personal Loan: ")
computesummary(sum0)

In [ ]:
computeaccuracy(y, y_p):
c = confusion_matrix(y, y_p).ravel()
tp, tn, fp, fn = c
acc = (tp + tn) / (tp + tn + fp + fn)
# print("False Positive: ", fp)
# print("False Negative: ", fn)
return acc, c

In [ ]:
liftChart(predicted, title='Decile Lift Chart', labelBars=True, ax=None, figsize=None):
groups = [int(10 * i / len(predicted)) for i in
          range(len(predicted))]
meanPercentile = predicted.groupby(groups).mean()
meanResponse = meanPercentile / predicted.mean()
meanResponse.index = (meanResponse.index + 1) * 10
ax = meanResponse.plot.bar(color='C0', ax=ax,
                           figsize=figsize)
ax.set_ylim(0, 1.12 * meanResponse.max() if labelBars else None)
ax.set_xlabel('Percentile')
ax.set_ylabel('Lift')
if title:
    ax.set_title(title)
if labelBars:
    for p in ax.patches:
        ax.annotate('{:.1f}'.format(p.get_height()),
                    (p.get_x(), p.get_height() + 0.1))
return ax

In [ ]:
y = ub_df["Personal Loan"]
X = ub_df.drop(["Personal Loan"], axis=1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4)
# Part A
# summary(ub_df)
lda_model = lda(X_train, y_train)
X_val = X_val.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)
y_pred = lda_model.predict(X_val)
prob = lda_model.predict_proba(X_val)
# Part B
accuracy,c_lda = computeaccuracy(y_val, y_pred)
output = "The Accuracy of model on Validation Set: {}".format(accuracy)
# print(output)
y_pred = pd.Series(y_pred, name="Prediciton")
result = X_val.join(y_pred)
result = result.join(y_val)
for i, r in result.iterrows():
    if r[11] != r[12]:
        if r[11] == 1:
            print("Observation: ", i, "Missclassified as Non: ", prob[i])
        else:
            print("Observation: ", i, "Missclassified as Acceptor: ", prob[i])
# Part C
probs = pd.Series(prob[:, 1]).sort_values(ascending=False)
liftChart(probs)
plt.show()
skplt.metrics.plot_cumulative_gain(y_val, prob)
plt.show()

# Part D
log_reg = logregress(X_train, y_train)
y_pred_log = log_reg.predict(X_val)
acc, c_lg = computeaccuracy(y_val, y_pred_log)
probs_log = log_reg.predict_proba(X_val)
prob_log = pd.Series(probs_log[:, 1]).sort_values(ascending=False)
liftChart(prob_log)
plt.show()
skplt.metrics.plot_cumulative_gain(y_val, probs_log)
plt.show()
print("Confusion Matrix for LDA: ", c_lda)
print("Confucion matrix for LG: ", c_lg)